In [ ]:
# this should get contextual embeddings for every word? for just all of the keywords that I'm looking at?
# https://medium.com/mlearning-ai/getting-contextualized-word-embeddings-with-bert-20798d8b43a4

import pandas as pd
import numpy as np
import torch
from transformers import BertModel, BertTokenizer
from tqdm import tqdm
import spacy
from pathlib import Path

nlp = spacy.blank("en")
nlp.add_pipe("sentencizer")

# https://www.semanticscholar.org/reader/0cbd7b946983b807735dc0e990c359ab57e2ad82
# these guys ^^ use ELMo instead of BERt b/c it requires less computationally
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# likely want to generate embeddings for each word and store all of those with the word, and its lemma
# then will use the lemma column to compare it to others that match
# after generating all of them, can use kNN as suggested by the Ukrainian paper
# can visualize them by using PCA projections of the word


# christian_df = pd.read_csv('final_project/christian_reddit.csv')
# lds_df = pd.read_csv('final_project/lds_reddit.csv')

# lds_texts = list(lds_df['text'].astype(str))
# christian_texts = list(christian_df['text'].astype(str))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [ ]:

christian_sentences = list(pd.read_csv(Path('christian_sents.csv'))['text'])
lds_sentences = list(pd.read_csv(Path('lds_sents.csv'))['text'].astype(str))

# print(christian_sentences)
# print(lds_sentences)

# christian_sentences = []
# lds_sentences = []

# # for each text, split the text by sentences and put those all into a list
# for doc in tqdm(nlp.pipe(lds_texts)):
#     try:
#       for sent in doc.sents:
#         lds_sentences.append(sent.text)
#     except Exception as e:
#       with open(Path("final_project/lds_sents_errors.txt"), "a") as f:
#         print(type(e), e, f"\n{doc}\n", file=f)

# sent_df = pd.DataFrame(lds_sentences, columns = ['text'])
# sent_df.to_csv(Path('final_project/lds_sents.csv'), index_label='id')


# for doc in tqdm(nlp.pipe(christian_texts)):
#   for sent in doc.sents:
#     christian_sentences.append(sent.text)

# sent_df = pd.DataFrame(christian_sentences, columns = ['text'])
# sent_df.to_csv(Path('final_project/christian_sents.csv'), index_label='id')

print(len(lds_sentences))
print(len(christian_sentences))


193047
389038


In [20]:
token_vecs_df = pd.DataFrame(columns=['token', 'embedding'])
tests = lds_sentences[:100]
last = 0
for i in range(1000, len(lds_sentences)-47, 1000):
  sents = lds_sentences[last:i]
  print(sents)
  last = i

  encoded_input = tokenizer.batch_encode_plus(sents, add_special_tokens=True,
                                              max_length=64, padding=True,
                                              truncation=True,
                                              return_attention_mask=True, return_tensors='pt',
                                              )

  print(encoded_input['input_ids'].shape)

  encoded_input.to(device)

  with torch.no_grad():
      output_embeddings = model(encoded_input['input_ids'], encoded_input['attention_mask'])

  hidden_states = output_embeddings[2]
  print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
  layer_i = 0
  print ("Number of batches:", len(hidden_states[layer_i]))
  batch_i = 0
  print(tokenizer.convert_ids_to_tokens(encoded_input['input_ids'][0]))
  print ("Number of tokens:", len(hidden_states[layer_i][batch_i])) # I'm going to somehow need to get the tokens attached back to their embeddings
  token_i = 0
  print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))

  token_embeddings = torch.stack(hidden_states, dim=0)
  token_embeddings.size()

  token_embeddings = token_embeddings.permute(1,2,0,3)
  token_embeddings.size()

  token_vecs_sum = []

  for batch_i, batch in enumerate(token_embeddings):
    print(batch.size())
    tokenized_text = tokenizer.convert_ids_to_tokens(encoded_input['input_ids'][batch_i])
    for token_i, token in enumerate(batch):
      sum_vec = torch.sum(token[-4:], dim=0)
      token_vecs_sum.append(sum_vec)
      if tokenized_text[token_i] != "[PAD]":
        vec_copy = sum_vec.cpu()
        token_vecs_df.loc[len(token_vecs_df.index)] = [tokenized_text[token_i], vec_copy]
  print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))

# token_vecs_df.to_csv(Path('lds_red_embeds.csv'))

token_vecs_df.to_csv(Path('lds_red_embeds.csv'))

Streaming output truncated to the last 5000 lines.
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([6

KeyboardInterrupt: ignored

In [21]:
token_vecs_df.to_csv(Path('lds_red_embeds.csv'))

Exception ignored in: <function _xla_gc_callback at 0x7be3731edcf0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 101, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


KeyboardInterrupt: ignored

In [28]:
token_vecs_df = pd.DataFrame(columns=['token', 'embedding'])
# tests = lds_sentences[:100]
last = 0
for i in range(1000, len(christian_sentences)-38, 1000):
  sents = [str(sent) for sent in christian_sentences[last:i]]
  print(sents)
  last = i

  if i==8000:
    break

  encoded_input = tokenizer.batch_encode_plus(sents, add_special_tokens=True,
                                              max_length=64, padding=True,
                                              truncation=True,
                                              return_attention_mask=True, return_tensors='pt',
                                              )

  encoded_input.to(device)
  with torch.no_grad():
      output_embeddings = model(encoded_input['input_ids'], encoded_input['attention_mask'])

  hidden_states = output_embeddings[2]
  # print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
  # layer_i = 0
  # print ("Number of batches:", len(hidden_states[layer_i]))
  # batch_i = 0
  # print(tokenizer.convert_ids_to_tokens(encoded_input['input_ids'][0]))
  # print ("Number of tokens:", len(hidden_states[layer_i][batch_i])) # I'm going to somehow need to get the tokens attached back to their embeddings
  # token_i = 0
  # print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))

  token_embeddings = torch.stack(hidden_states, dim=0)
  token_embeddings.size()

  token_embeddings = token_embeddings.permute(1,2,0,3)
  token_embeddings.size()

  token_vecs_sum = []
  for batch_i, batch in enumerate(token_embeddings):
    print(batch.size())
    tokenized_text = tokenizer.convert_ids_to_tokens(encoded_input['input_ids'][batch_i])
    # zip them together for each batch here, which should correspond to a sentence
    for token_i, token in enumerate(batch):
      sum_vec = torch.sum(token[-4:], dim=0)
      token_vecs_sum.append(sum_vec)
      if tokenized_text[token_i] != "[PAD]":
        vec_copy = sum_vec.cpu()
        token_vecs_df.loc[len(token_vecs_df.index)] = [tokenized_text[token_i], vec_copy]
  print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))

token_vecs_df.to_csv(Path('christian_red_embeds.csv'))

Streaming output truncated to the last 5000 lines.
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([64, 13, 768])
torch.Size([6

OSError: ignored

In [29]:
token_vecs_df.to_csv(Path('christian_red_embeds.csv'))